In [1]:
import requests 
import json 
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
import pandas as pd 
import numpy as np 
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 50)

In [3]:
# Insert paths here
data_path = "C:/Users/Joel/Downloads/Kickstarter_2021-04-15T03_20_08_451Z/Kickstarter.csv"
driver_path = "C:/Users/Joel/Downloads/chromedriver_win32/chromedriver.exe"

driver = webdriver.Chrome(driver_path)
df = pd.read_csv(data_path)

In [4]:
def proj_url(df, index):
    tmp = df.iloc[index]
    # is stored as a json type 
    url = json.loads(tmp['urls'])['web']['project']
    return url 

In [5]:
def load_text_url(url):
    req = requests.get(url)
    html = req.content
    soup = BeautifulSoup(html, features="html.parser")
    # kill all script and style elements
    return soup 
    for script in soup(["script", "style"]):
        script.extract()    # rip it out
    # get text
    text = soup.get_text()
    return soup.find_all(text=True)

In [6]:
def extract_proj_descrip(url):
    # i think descriptions are pulled up w/ some java script bs so we need to load in driver
    driver.get(url)
    html = driver.page_source
    # load html 
    soup = BeautifulSoup(html, "lxml")
    proj = soup.find("section", {"class": "project-content"})
    return proj, proj.find_all(text=True)

In [42]:
def shorten_list(wrds):
    # words signal end of project description 
    beg = 'Story'
    # Set endings
    end1 = 'Check out the FAQ'
    end2 = 'Report this project to Kickstarter'
    end3 = 'Select this reward'
    end4 = 'Learn about accountability on Kickstarter'
    endings = [end1, end2, end3, end4]
    
    beg_ind = 0
    
    for i in range(len(wrds)):
        if beg_ind == 0 and beg in wrds[i]:
            beg_ind = i
        elif beg_ind != 0 and any(x in wrds[i] for x in endings):
            end_ind = i
            return wrds[beg_ind:end_ind]
    
    print("no end")
    #beg = np.where(np.array(wrds)=='Story')[0][-1]
    #end1 = np.where(np.array(wrds)=='Check out the FAQ')[0][0]
    #end2 = np.where(np.array(wrds)=='Report this project to Kickstarter')[0][0]
    #end3 = np.where(np.array(wrds)=='Select this reward')[0][0]
    #end4 = np.where(np.array(wrds)=='Learn about accountability on Kickstarter')[0][0]
    #min_ind = min(end1, end2, end3, end4)

In [8]:
# TEMPORARY - DELETE LATER
# Will loop through all URLs in future
tmp = df.iloc[0]
tmp_url = json.loads(tmp['urls'])['web']['project']

In [9]:
p = extract_proj_descrip(tmp_url)
print(p)

(<section class="js-project-content js-project-description-content project-content" data-project-state="failed">
<div class="NS_projects__description_section m-auto">
<div class="bg-white grid-container pr3-sm py8">
<div class="row">
<div class="col col-8 description-container">
<div class="px3">
<div data-slug="939819973/crimpcoffee-good-coffee-good-times" id="react-campaign"><section class="project-content"><div class="row"><nav class="col col-2 important-stick-md hide block-lg pb4 pl0 t20-md"><div><a class="type-13 text-left ls0_2px text-decoration-none-important hover-ksr-green-500 block relative text-uppercase pb3 border-bottom dark-grey-500" href="#story">Story</a></div><div><a class="type-13 text-left ls0_2px text-decoration-none-important hover-ksr-green-500 block relative text-uppercase py3 dark-grey-500" href="#risks-and-challenges">Risks</a></div></nav><div class="col col-12 grid-col-10-lg"><div class="max-w113 m-auto"><div><div><div><div class="mb3" id="story"><h3 class="no

In [28]:
p, ptxt = extract_proj_descrip(proj_url(df, 2))
print(p)
print(ptxt)

<section class="js-project-content js-project-description-content project-content" data-project-state="failed">
<div class="NS_projects__description_section m-auto">
<div class="bg-white grid-container pr3-sm py8">
<div class="row">
<div class="col col-8 description-container">
<div class="px3">
<div data-slug="dreamcatchercomics/dreamcatcher-games" id="react-campaign"><section class="project-content"><div class="row"><nav class="col col-2 important-stick-md hide block-lg pb4 pl0 t20-md"><div><a class="type-13 text-left ls0_2px text-decoration-none-important hover-ksr-green-500 block relative text-uppercase pb3 border-bottom dark-grey-500" href="#story">Story</a></div><div><a class="type-13 text-left ls0_2px text-decoration-none-important hover-ksr-green-500 block relative text-uppercase py3 dark-grey-500" href="#risks-and-challenges">Risks</a></div></nav><div class="col col-12 grid-col-10-lg"><div class="max-w113 m-auto"><div><div><div><div class="mb3" id="story"><h3 class="normal mb3

In [44]:
for i in range(5):
    p, ptxt = extract_proj_descrip(proj_url(df, i))
    print(shorten_list(ptxt))

['Story', 'Risks', 'Story', '\n', 'CrimpCoffee:\n', 'Who are we? \n\xa0', '\n', "\xa0 \xa0 \xa0We are the crazy brainchild conceived during a spontaneous bouldering trip on memorial weekend in Leavenworth, Washington 2017. Armed with my trusty Cherokee, a rental crash pad, and a Jetboil, my friend and I wondered Icicle Canyon from one boulder field to another for four days. The morning brew quickly became an essential tool for long back to back days of hard climbing and hiking. \nWhile swinging in a hammock soaking up Leavenworth's beauty, I realized I want to be more involved with our beautiful land, and its amazing community that loves to explore it just as much as I do. And what do we love the most in the pacific northwest? Good food, good coffee,and good times, isn’t that all you really need in life?\n\xa0", '\xa0 \xa0 \xa0The same day being back from Leavenworth, the Cherokee went up for sale. The following weekend, it was sold to a new home and a 2000 Chevy Astro AWD Cargo van wa

['Story', 'Risks', 'Story', 'Here at ', 'Dreamcatcher Comics', ' we are a start up company about to produce issue one of our comic The Gatekeepers.', 'We are looking at creating a rollerball game and other games/apps. We are currently looking for funding to cover such things as Software etc.\xa0', 'This Game will be a puzzle game using our very own Hunter as your first main character with more unlockable characters thoughout the game. at first your main challenge will be battling though the many levels and defeating terrain. Afterwards your face with defeating the Alptraum, The Gatekeepers most feared enimies. they will cause you all kinds of trouble by setting you time limits and throwing obsticles in your way.', 'Our main aim for this game will be to have both joystick like controls and also Tilt controls for smart phones. We will be bringing this Game to both android and iphone and all relating tablets. We also hope to bring out a pc version of the game to allowing all kinds of game

['Story', 'Risks', 'Story', "We're a women-run and mission-driven food\xa0brand handcrafting flavorful and functional condiments for everyday use. We source only organic ingredients directly from small independent farms or cooperatives all over the world blending with adaptogenic mushrooms to create our magic pantry staples.", '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', '\n', 'Over the first few weeks of the pandemic quarantine in New York, I began missing my favorite restaurant foods. The staples were chili oil and flavorful spices. I ordered everything that existed on the market and even looked for recipes to create my own, but nothing tasted right.\xa0', '\n', 'After being diagnosed with polycystic ovary syndrome (PCOS), one of the most common hormone disorders in women, I became extremely health conscious.', ' I was determined to figure out how to manage PCOS through my diet and have been hyper aware of everything I put in my body ever since. Using passion and love 

In [31]:
[w for w in ptxt if w != '\n']

['Story',
 'Risks',
 'Story',
 'Here at ',
 'Dreamcatcher Comics',
 ' we are a start up company about to produce issue one of our comic The Gatekeepers.',
 'We are looking at creating a rollerball game and other games/apps. We are currently looking for funding to cover such things as Software etc.\xa0',
 'This Game will be a puzzle game using our very own Hunter as your first main character with more unlockable characters thoughout the game. at first your main challenge will be battling though the many levels and defeating terrain. Afterwards your face with defeating the Alptraum, The Gatekeepers most feared enimies. they will cause you all kinds of trouble by setting you time limits and throwing obsticles in your way.',
 'Our main aim for this game will be to have both joystick like controls and also Tilt controls for smart phones. We will be bringing this Game to both android and iphone and all relating tablets. We also hope to bring out a pc version of the game to allowing all kinds

In [52]:
tst = [w for w in ptxt if w != '\n']

In [62]:
tst

['\nSpawns of the Great Rot - Printable stl 3d miniatures\n',
 '\nSvendborg, Denmark\n',
 '\nTabletop Games\n',
 'DKK 104,176',
 '\npledged of ',
 'DKK 7,500',
 ' goal\n',
 '\n454\n',
 '\nbackers\n',
 'Story',
 'Risks',
 'Environmental commitments',
 'Story',
 "Before the great rot, death was permanent. Unfortunately, death for those who were affected by the Soul Rot wasn't the end.",
 'The Soul Rot forced the soul to live on after the body gave up, condemning the soul to live forever in the state it left the body. These poor souls are known as Geists.',
 'Welcome to the first campaign by Fleshcraft studio!',
 '\n \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Working on a scary campaign for your DnD group? Or wanna expand the looks of your undead armies? Well, “unfortunately” for you, the ghosts have come to haunt you! In this campaign, you will receive a set of terrifying digital 3D ghosts in the .stl format. ',
 '\n \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\x

In [13]:
shorten_list(ptxt)

['Story',
 'Here at ',
 'Dreamcatcher Comics',
 ' we are a start up company about to produce issue one of our comic The Gatekeepers.',
 'We are looking at creating a rollerball game and other games/apps. We are currently looking for funding to cover such things as Software etc.\xa0',
 'This Game will be a puzzle game using our very own Hunter as your first main character with more unlockable characters thoughout the game. at first your main challenge will be battling though the many levels and defeating terrain. Afterwards your face with defeating the Alptraum, The Gatekeepers most feared enimies. they will cause you all kinds of trouble by setting you time limits and throwing obsticles in your way.',
 'Our main aim for this game will be to have both joystick like controls and also Tilt controls for smart phones. We will be bringing this Game to both android and iphone and all relating tablets. We also hope to bring out a pc version of the game to allowing all kinds of gamers to play.'